# Project 1 for LING413 
##### Part 1

Imports:

In [1]:
import codecs
import pandas as pd
import random
from sklearn import feature_extraction
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
import pickle

Loading and Formatting Data:

In [2]:
file = "Twitter_by_country.gz"

n = 1140248 # num samples in file
s = 200000 # num samples to use

skip = sorted(random.sample(range(n), n-s))

data_df = pd.read_csv(file, sep = ",", header = None, skiprows=skip)
data_df.drop(0, axis = 1, inplace = True)
data_df.drop(0, axis = 0, inplace = True)
data_df.columns = ["Country", "Tweet"]

print(data_df.head())
print(data_df.shape)

     Country                                              Tweet
1  Australia  all the evidence anyone should need to know th...
2  Australia  over the birds of the heavens and over the liv...
3  Australia  biggest present under the tree is for me from ...
4  Australia  my boyfriend got me a diamond bracelet and his...
5  Australia  should spend more time covering your own ass b...
(200000, 2)


In [3]:
features = feature_extraction.text.CountVectorizer(input='content', 
                                                encoding='utf-8',
                                                lowercase=True, 
                                                tokenizer = None,
                                                ngram_range=(1, 1), 
                                                analyzer='word',
                                                max_features=10000, #This sets vocab size
                                                )

#Sklearn first fits then transforms
features.fit(data_df.loc[:,"Tweet"])
data_x = features.transform(data_df.loc[:,"Tweet"])
data_y = data_df.loc[:,"Country"]

Train Classifier:

In [4]:
x_train, x_test, y_train, y_test = train_test_split(data_x, data_y, test_size=0.10, shuffle=True)

print("Train shape", x_train.shape, y_train.shape)
print("Test shape", x_test.shape, y_test.shape)


cls = LogisticRegression(max_iter=4000)
cls.fit(x_train, y_train)
print(cls)

Train shape (180000, 10000) (180000,)
Test shape (20000, 10000) (20000,)
LogisticRegression(max_iter=4000)


Testing:

In [5]:
y_pred = cls.predict(x_test)
report = classification_report(y_test, y_pred)

with codecs.open("unigram_tweet_classification_results.txt", "w", encoding = "utf-8") as f:
    f.write(report)

print(report)

                precision    recall  f1-score   support

     Australia       0.88      0.88      0.88      3350
       Ireland       0.91      0.91      0.91      3386
   New_Zealand       0.86      0.86      0.86      3125
  South_Africa       0.95      0.95      0.95      3404
United_Kingdom       0.88      0.88      0.88      3367
 United_States       0.92      0.92      0.92      3368

      accuracy                           0.90     20000
     macro avg       0.90      0.90      0.90     20000
  weighted avg       0.90      0.90      0.90     20000



Save Model for Later:

In [6]:
with open("unigram_tweet_classification_model.p", "wb") as handle:
    pickle.dump(cls, handle)